In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from tqdm import tqdm

In [38]:
output_dir = Path.home() / 'Desktop' / 'disk1' / 'ecephys_data'

cache = EcephysProjectCache.from_warehouse(manifest=output_dir / 'manifest.json')

In [39]:
probe_df = cache.get_probes()
print(len(probe_df))
probe_df

332


,ecephys_session_id,lfp_sampling_rate,name,phase,sampling_rate,has_lfp_data,unit_count,channel_count,ecephys_structure_acronyms
id,,,,,,,,,
729445648,719161530,1249.998642,probeA,3a,29999.967418,True,87,374,"[APN, LP, MB, DG, CA1, VISam, nan]"
729445650,719161530,1249.996620,probeB,3a,29999.918880,True,202,368,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan]"
729445652,719161530,1249.999897,probeC,3a,29999.997521,True,207,373,"[APN, NOT, MB, DG, SUB, VISp, nan]"
729445654,719161530,1249.996707,probeD,3a,29999.920963,True,93,358,"[grey, VL, CA3, CA2, CA1, VISl, nan]"
729445656,719161530,1249.999979,probeE,3a,29999.999500,True,138,370,"[PO, VPM, TH, LP, LGd, CA3, DG, CA1, VISal, nan]"
...,...,...,...,...,...,...,...,...,...
848037570,847657808,1250.009761,probeB,PXI,30000.234268,True,148,383,"[APN, MB, NOT, DG, ProS, VISpm, nan]"
848037572,847657808,1250.012584,probeC,PXI,30000.302018,True,89,383,"[DG, POST, SUB, VISp, nan]"
848037574,847657808,1250.011776,probeD,PXI,30000.282632,True,155,383,"[DG, CA3, CA1, VISl, nan]"


In [40]:
unique_structures = []
for probe_structures in cache.get_probes()['ecephys_structure_acronyms']:
    for item in probe_structures:
        if item not in unique_structures:
            unique_structures.append(item)
print(unique_structures)

['APN', 'LP', 'MB', 'DG', 'CA1', 'VISam', nan, 'TH', 'Eth', 'POL', 'VISpm', 'NOT', 'SUB', 'VISp', 'grey', 'VL', 'CA3', 'CA2', 'VISl', 'PO', 'VPM', 'LGd', 'VISal', 'VISrl', 'HPF', 'VIS', 'SGN', 'IGL', 'VPL', 'IntG', 'PoT', 'MGm', 'LGv', 'VISmma', 'SCig', 'PPT', 'ProS', 'POST', 'MGv', 'MGd', 'ZI', 'VISmmp', 'MRN', 'LD', 'OP', 'VISli', 'PP', 'PIL', 'LT', 'RPF', 'PF', 'SCiw', 'PRE', 'CP', 'COAa', 'SCop', 'COApm', 'OLF', 'BMAa', 'SCsg', 'SCzo', 'RT']


In [43]:
def get_probes_from_specified_areas(areas):
    probe_id_array = np.array([])
    for area in areas:
        if area not in unique_structures:
            print(f'THERE ARE NOT RECORDINGS FROM "{area}" FOUND.')
        else:
            print(f'AREA "{area}" FOUND. SELECTING PROBES WITH LFP DATA.')
            new_id_array = probe_df[(probe_df['ecephys_structure_acronyms'].apply(lambda x: area in x)) & (probe_df['has_lfp_data'] == True)].index.to_list()
            print(f'{len(new_id_array)} FOUND.')
            probe_id_array = np.concatenate((probe_id_array,new_id_array),axis=0).astype(int)
    return probe_id_array

In [44]:
selected_probe_ids = get_probes_from_specified_areas(['VISp'])
selected_probe_ids

AREA "VISp" FOUND. SELECTING PROBES WITH LFP DATA.
56 FOUND.


array([729445652, 733744649, 756781557, 757904513, 757904550, 757968493,
       757984820, 757988391, 760213145, 760640039, 760640049, 760640090,
       760642628, 760647909, 768908152, 768908585, 768909182, 769322716,
       769322749, 769322793, 769322806, 769322827, 769322856, 769325786,
       770930073, 773462993, 773463017, 773549848, 773592320, 773621942,
       773654728, 792586842, 792586881, 792602654, 792607549, 792623921,
       792626847, 792645497, 792676158, 800036200, 805002031, 805008604,
       805124806, 805579703, 805579741, 810753199, 810755801, 810758781,
       822645897, 832129154, 832810578, 836943717, 837761710, 868297129,
       868929138, 848037572])

In [45]:
# Download session metadata and LFP data for every selected probe
for probe_id in tqdm(selected_probe_ids):
    session = cache.get_session_data(cache.get_probes()[cache.get_probes().index==probe_id]['ecephys_session_id'].item())
    lfp = session.get_lfp(probe_id)

  0%|          | 0/56 [00:00<?, ?it/s]WARNING:root:downloading a 2929.156MiB file from http://api.brain-map.org//api/v2/well_known_file_download/1026124034


Downloading:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

/home/ikharitonov/anaconda3/envs/allendata/lib/python3.7/site-packages/allensdk/api/warehouse_cache/caching_utilities.py:127: UserWarning: retrying fetch (1 tries remaining)
  warnings.warn(retry_message)


Downloading:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

  0%|          | 0/56 [40:04<?, ?it/s]


Timeout: Download took 1200.0897001830017 seconds, but timeout was set to 1200